In [1]:
### SLICER
## setup mean model
## align model center to CT center
## optionally invert axes
## scale model to 0.5*CT_ROI
## define manual landmarks on CT
## optimize to match manual landmarks


### GREY LEVEL MODEL
## align SSM sample to CT-segmented-model
## define landmarks
## extract profiles
## compute covariance matrix

# DEBUG

In [203]:
from extract_profile import *
%matplotlib

patientName = "V2"

def convertRasToIjk(points):
    points = np.c_[points, np.ones(points.shape[0])]
    pointsIjk = np.einsum('ab,ib->ia', RASToIjk, points)[:,:-1]
    return pointsIjk

inputVolumePath = inputVolumePathTemplate.format(patientName)
inputVolumeNib = nib.load(inputVolumePath)
print("Loading nifti...", end=" ")
inputVolumeNumpy = inputVolumeNib.get_fdata()
print("done!")
ijkToRASMatrix = inputVolumeNib.affine
RASToIjk = np.linalg.inv(ijkToRASMatrix)

# define interpolation method for the image volume
ii, jj, kk = [np.arange(0,inputVolumeNumpy.shape[i]) for i in range(3)]
interpolator = scipy.interpolate.RegularGridInterpolator(
    (ii, jj, kk),
    inputVolumeNumpy,
    method='linear',
    bounds_error=False,
    fill_value=0.,
)
SSMModelPath = SSMModelPathTemplate.format(patientName)
SSMModel = pv.read(SSMModelPath)
LPSToRASMatrix = np.diag([-1,-1,1,1])
if convertSSMModelsToRAS:
    SSMModel = SSMModel.transform(LPSToRASMatrix)
origModelPath = origModelPathTemplate.format(patientName)
origModel = pv.read(origModelPath)

if convertSSMModelsToRAS:
    origModel = origModel.transform(LPSToRASMatrix)
SSMTransformed = ICP(SSMModel, origModel)
SSMTransformed = SSMTransformed.compute_normals(cell_normals=False)

#SSMTransformed.save('SSMTrasformed.vtp')

# # define sample points
# distances = scipy.spatial.distance.pdist(SSMTransformed.points , metric='euclidean')
# distances = scipy.spatial.distance.squareform(distances, force='tomatrix', checks=True)
# nSamplePoints = 500
# samplePointsIdxs = getGreedyPerm(distances, nSamplePoints)[0]
# samplePoints = SSMTransformed.points[samplePointsIdxs]
# np.savetxt('samplePoints.txt', samplePointsIdxs, fmt="%i", delimiter=',')
# pv.PolyData(samplePoints).save('samplePointsForA2Model.vtp')

samplePointsIdxs = np.loadtxt('samplePoints.txt', dtype=int)
samplePoints = SSMTransformed.points[samplePointsIdxs]
sampleNormals = SSMTransformed['Normals'][samplePointsIdxs]

def computeProfilePoints(point, direction):
    t = np.arange(-profileLen/2., (profileLen)/2 + profileSpacing, profileSpacing)
    # (t.shape[0], nSamplePoints, 3)
    return point + np.einsum('i,kj->ikj', t, direction)

# compute coordinates of sampling points (near the model's surface)
profilePoints = computeProfilePoints(samplePoints, sampleNormals)
# convert sampling points coordinate to IJK space to sample the image
pointsIjk = convertRasToIjk(profilePoints.reshape(-1,3))
profiles = interpolator(pointsIjk).reshape(profilePoints.shape[:-1])

profiles_norm_grad = np.gradient(profiles, profileSpacing, axis=0)
profiles_norm_grad = profiles_norm_grad / profiles_norm_grad.std(axis=0)

if not osp.isdir(profilesDir):
    os.makedirs(profilesDir)
profilePathTemplate = osp.join(profilesDir, "{}.txt")
np.savetxt(profilePathTemplate.format(patientName), profiles_norm_grad.T.reshape(-1), fmt='%.5f')

# plots
profilesPlotsDir = osp.join(profilesDir, "plots")
if not osp.isdir(profilesPlotsDir):
    os.makedirs(profilesPlotsDir)
plotsPathTemplate = osp.join(profilesPlotsDir, "{}.png")
if plots:
    t = np.arange(-profileLen/2., (profileLen)/2 + profileSpacing, profileSpacing)
    plt.errorbar(t, profiles_norm_grad.mean(-1), yerr=profiles_norm_grad.std(-1))
    plt.xlabel("displacement from surface [mm]")
    plt.ylabel("normalized gradient")
    plt.title("{} profiles mean".format(patientName))
    #plt.savefig(plotsPathTemplate.format(patientName), dpi=80)
    #plt.close()
    plt.show()

In [213]:
origModel.save('orig.vtp')
SSMTransformed.save('transformed.vtp')
#pv.PolyData(convertRasToIjk(SSMTransformed.points)).save('SSMTransformed_IJKSpace.vtp')

# Covariance and Mahalanobis distance

In [1]:
import compute_profiles_covariance as Covariance

In [2]:
profiles = Covariance.loadProfiles()

In [3]:
cov, mean, std = Covariance.get_covariance_object(profiles, load=True)

In [15]:
cov.mahalanobis((profiles - mean) / std)

array([ 94.53223648,  95.11225354, 103.86733194,  94.85320516,
       109.72711813,  95.68317252, 101.17633584,  94.84570704,
        94.76644382,  96.62729175, 105.63122903,  91.28632643,
       100.9914105 , 101.60277366, 102.22377276,  91.25302704,
       104.10853368, 106.21893987, 103.94872908, 103.81646305,
       104.98721126,  99.3242734 ,  91.63005114,  88.25961331,
        77.69849354, 101.84943665,  86.96715198,  92.06248993,
        90.12448427, 103.69803474,  92.11975173,  97.92437079,
        97.24616755,  94.71898841,  91.3031026 ,  82.46432302,
        97.75167547,  92.721234  , 100.3363223 ,  86.6782274 ,
        92.19193479,  87.57091732, 106.6722439 ,  90.74274838,
        92.3138823 ])

# registration optimization

In [ ]:
parameters = np.zeros